In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,57116
2,Huelva,Confirmados PDIA 14 días,6513
3,Huelva,Tasa PDIA 14 días,"1269,1700606036986"
4,Huelva,Confirmados PDIA 7 días,3345
5,Huelva,Tasa PDIA 7 dias,"651,8307773252528"
6,Huelva,Total Confirmados,57326
7,Huelva,Curados,47754
8,Huelva,Fallecidos,427


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  57116.0


/tmp/ipykernel_3443/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  16286.0


/tmp/ipykernel_3443/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_3443/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3443/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3443/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 143 personas en los últimos 7 días 

Un positivo PCR cada 69 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,57116.0,6513.0,3345.0,513170.0,651.830777,1269.170061,399.0
Huelva-Costa,33602.0,3782.0,1813.0,289548.0,626.148342,1306.173760,182.0
Condado-Campiña,17243.0,1578.0,912.0,156231.0,583.750984,1010.042821,120.0
Huelva (capital),16286.0,2078.0,1003.0,143837.0,697.317102,1444.690865,119.0
Sierra de Huelva-Andévalo Central,5826.0,1140.0,613.0,67391.0,909.617011,1691.620543,93.0
Palos de la Frontera,1412.0,154.0,88.0,11742.0,749.446432,1311.531255,21.0
Aljaraque,2180.0,331.0,146.0,21474.0,679.891962,1541.398901,21.0
Moguer,2405.0,177.0,85.0,21867.0,388.713587,809.438881,18.0
Minas de Riotinto,261.0,56.0,36.0,3812.0,944.386149,1469.045121,13.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Silvestre de Guzmán,75.0,49.0,11.0,614.0,1791.530945,7980.456026,0.0
Fuenteheridos,45.0,22.0,7.0,709.0,987.306065,3102.961918,0.0
Galaroza,105.0,41.0,27.0,1382.0,1953.690304,2966.714906,3.0
Valverde del Camino,1291.0,334.0,155.0,12750.0,1215.686275,2619.607843,9.0
Campofrío,80.0,18.0,16.0,713.0,2244.039271,2524.544180,3.0
Puerto Moral,25.0,7.0,4.0,278.0,1438.848921,2517.985612,0.0
Escacena del Campo,300.0,54.0,24.0,2287.0,1049.409707,2361.171841,6.0
Aracena,1035.0,170.0,69.0,8255.0,835.857056,2059.357965,12.0
Cumbres Mayores,217.0,35.0,18.0,1749.0,1029.159520,2001.143511,6.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Puebla de Guzmán,226.0,43.0,9.0,3092.0,291.073739,1390.685640,1.0,0.209302
San Silvestre de Guzmán,75.0,49.0,11.0,614.0,1791.530945,7980.456026,0.0,0.224490
Santa Bárbara de Casa,53.0,4.0,1.0,1043.0,95.877277,383.509108,0.0,0.250000
Fuenteheridos,45.0,22.0,7.0,709.0,987.306065,3102.961918,0.0,0.318182
Higuera de la Sierra,92.0,15.0,5.0,1291.0,387.296669,1161.890008,1.0,0.333333
Alosno,317.0,49.0,17.0,3933.0,432.240020,1245.868294,0.0,0.346939
Aracena,1035.0,170.0,69.0,8255.0,835.857056,2059.357965,12.0,0.405882
Alájar,50.0,14.0,6.0,759.0,790.513834,1844.532279,1.0,0.428571
Ayamonte,2553.0,332.0,143.0,21104.0,677.596664,1573.161486,13.0,0.430723
